In [5]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [6]:
llm = Ollama(model="gemma:7b")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
import re

def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'CREATE' in line or 'MATCH' in line or 'CONNECT' in line:
            cypher_list.append(line)
            
    return cypher_list

## CoT Knowledge Grapg Creation

In [8]:
import pandas as pd
import numpy as np
cypher_list = list()
contexts = pd.read_csv('/Users/mertakcay/Projects/KnowledgeGraphBuilder/context.csv')
context_outputs = list()
for context in contexts['context']:
    prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a ontologist. You build knowledge graph from plain text. \n 
     Task 1: Extract Entities from plain text as possible as atomic. \n
     Task 2: Define Cypher Node Type to each Entity.\n
     Task 3: Add features of each Entity from plain text. \n
     Task 4: Extract relationship between Entities from plain text as possible as atomic. \n\n
     Task 5: Convert all outputs to Neo4j Cypher Creation Format"""),
    ("system", "Let's think step by step."),
    ("user", "{input}")
    ])
    chain = prompt | llm 
    output = chain.invoke({"input": context})
    print(output)
    print('--------------------------------')
    cypher_scripts = extract_cypher_scripts(output.replace('`','"'))
    context_outputs.append(cypher_scripts)
    

**Task 1: Extract Entities from Plain Text**

**Entities:**
- Beyoncé Giselle Knowles-Carter
- Houston, Texas
- Destiny's Child
- Mathew Knowles
- Dangerously in Love
- Grammy Awards

**Task 2: Define Cypher Node Type to Each Entity**

**Node Type:**
- **Person:** Beyoncé Giselle Knowles-Carter, Mathew Knowles
- **Location:** Houston, Texas
- **Group:** Destiny's Child
- **Album:** Dangerously in Love

**Task 3: Add Features of Each Entity from Plain Text**

**Features:**
- **Person:** Born in 1981, singer, songwriter, record producer, actress
- **Location:** Born and raised in Houston, Texas
- **Group:** Lead singer, one of the world's best-selling girl groups
- **Album:** Released in 2003, earned five Grammy Awards, featured number-one singles

**Task 4: Extract Relationship Between Entities from Plain Text**

**Relationships:**
- **Born in:** Beyoncé Giselle Knowles-Carter was born in Houston, Texas.
- **Led singer of:** Destiny's Child was led by Beyoncé Giselle Knowles-Carter.
- *

KeyboardInterrupt: 